In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from zipfile import ZipFile
# Create a ZipFile Object and load sample.zip in it
with ZipFile('/content/drive/MyDrive/alldata.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/content/temp')

In [5]:
#@title
# import os
# os.mkdir("/content/jpgs")
# os.mkdir("/content/jpgs/meningioma")
# os.mkdir("/content/jpgs/glioma")

In [6]:
#@title
# from PIL import Image
# import numpy as np
# import h5py as h5
# import glob
# import os
# from skimage.util import img_as_ubyte
# import cv2

# path = '/content/temp/imagedata/*.mat'   
# files=glob.glob(path)

# for file in files:
#   f=h5.File(file,'r')
#   label = int(f['/cjdata/label'][0][0])

#   if label == 3:
#     os.remove(file)

#   elif label == 1:
#     img = np.array(f['/cjdata/image'])
#     img = img_as_ubyte(img)
#     img = img/255
#     img = img_as_ubyte(img)
#     img = cv2.medianBlur(img,3)

#     im = Image.fromarray(img)
#     file_name, ext = os.path.splitext(file)
#     fff = file_name.split('/')
#     im.save("/content/jpgs/meningioma/"+fff[-1]+".jpg")

#   elif label == 2:
#     img = np.array(f['/cjdata/image'])
#     img = img_as_ubyte(img)
#     img = cv2.medianBlur(img,3)
    
#     im = Image.fromarray(img)
#     file_name, ext = os.path.splitext(file)
#     fff = file_name.split('/')
#     im.save("/content/jpgs/glioma/"+fff[-1]+".jpg")

In [7]:
from PIL import Image
import numpy as np
import h5py as h5
import glob
import os
from skimage.util import img_as_ubyte, img_as_float
import cv2

import pandas as pd
import glob
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from skimage.feature import greycomatrix, greycoprops

import pywt

features= pd.DataFrame(columns=[])
labels = []

path = '/content/temp/imagedata/*.mat'   
files=glob.glob(path)

for file in files:
  f=h5.File(file,'r')
  label = int(f['/cjdata/label'][0][0])

  if label == 3:
    os.remove(file)
  else:
      labels.append(label)

      img = np.array(f['/cjdata/image'])
      img = cv2.resize(img, (256,256))
      img = img_as_ubyte(img)


      mean=img.mean()  #feature 3                              
      var=img.var()   #feature 4                               
      flat_image = img.flatten()                                  
      skewness = skew(flat_image, bias = False)                  #from scipy.stats import skew, kurtosis 
      kurt = kurtosis(flat_image, bias = False)

      glcm1 = greycomatrix(img, distances=[1], angles=[0], symmetric=True, normed=True)
      contrast1 = greycoprops(glcm1, 'contrast')[0][0]         # feature 5
      energy1 = greycoprops(glcm1,'energy')[0][0]              # feature 6
      ASM1 = greycoprops(glcm1, 'ASM')[0][0]                   # feature 7
      homogeneity1 = greycoprops(glcm1, 'homogeneity')[0][0]   # feature 8
      correlation1 = greycoprops(glcm1, 'correlation')[0][0]

      glcm2 = greycomatrix(img, distances=[1], angles=[np.pi/4] , symmetric=True, normed=True)
      contrast2 = greycoprops(glcm2, 'contrast')[0][0]         # feature 5
      energy2 = greycoprops(glcm2,'energy')[0][0]              # feature 6
      ASM2 = greycoprops(glcm2, 'ASM')[0][0]                   # feature 7
      homogeneity2 = greycoprops(glcm2, 'homogeneity')[0][0]   # feature 8
      correlation2 = greycoprops(glcm2, 'correlation')[0][0]

      glcm3 = greycomatrix(img, distances=[1], angles=[np.pi/2], symmetric=True, normed=True)
      contrast3 = greycoprops(glcm3, 'contrast')[0][0]         # feature 5
      energy3 = greycoprops(glcm3,'energy')[0][0]              # feature 6
      ASM3 = greycoprops(glcm3, 'ASM')[0][0]                   # feature 7
      homogeneity3 = greycoprops(glcm3, 'homogeneity')[0][0]   # feature 8
      correlation3 = greycoprops(glcm3, 'correlation')[0][0]

      glcm4 = greycomatrix(img, distances=[1], angles=[(3*np.pi)/4], symmetric=True, normed=True)
      contrast4 = greycoprops(glcm4, 'contrast')[0][0]         # feature 5
      energy4 = greycoprops(glcm4,'energy')[0][0]              # feature 6
      ASM4 = greycoprops(glcm4, 'ASM')[0][0]                   # feature 7
      homogeneity4 = greycoprops(glcm4, 'homogeneity')[0][0]   # feature 8
      correlation4 = greycoprops(glcm4, 'correlation')[0][0]

      mean_contrast = (contrast1 + contrast2 + contrast3 + contrast4)/4
      mean_energy = (energy1+ energy2+ energy3+ energy4)/4
      mean_ASM = (ASM1+ ASM2+ ASM3+ ASM4)/4
      mean_homogeneity = (homogeneity1+ homogeneity2+ homogeneity3+ homogeneity4)/4
      mean_correlation = (correlation1+ correlation2+ correlation3+ correlation4)/4

      range_contrast = max(contrast1, contrast2, contrast3, contrast4) -min(contrast1, contrast2, contrast3, contrast4)
      range_energy = max(energy1, energy2, energy3, energy4) - min(energy1, energy2, energy3, energy4)
      range_ASM = max(ASM1, ASM2, ASM3, ASM4) - min(ASM1, ASM2, ASM3, ASM4)
      range_homogeneity = max(homogeneity1, homogeneity2, homogeneity3, homogeneity4) - min(homogeneity1, homogeneity2, homogeneity3, homogeneity4)
      range_correlation = max(correlation1, correlation2, correlation3, correlation4) - min(correlation1, correlation2, correlation3, correlation4)
    
      features = features.append({
          'Mean': mean,
          'variance':var, 
          'skewness': skewness, 
          'kurtosis': kurt,
          'Contrast': mean_contrast,
          'Energy': mean_energy,
          'ASM': mean_ASM,
          'Homogeneity': mean_homogeneity,
          'Correlation': mean_correlation,
          'Range_Contrast': range_contrast,
          'Range_Energy': range_energy,
          'Range_ASM': range_ASM,
          'Range_Homogeneity': range_homogeneity,
          'Range_Correlation': range_correlation}, ignore_index=True)



In [8]:
#@title

# import pandas as pd
# import glob
# import matplotlib.pyplot as plt
# from scipy.stats import skew, kurtosis
# from skimage.feature import greycomatrix, greycoprops

# features= pd.DataFrame(columns=[])
# labels = []

# path = '/content/jpgs/glioma/*.jpg'                          #import glob
# files=glob.glob(path)

# for file in files:
#   img = plt.imread(file) 
#   img = np.array(img)                                  # import matplotlib.plt as plt
#   mean=img.mean()  #feature 3                              
#   var=img.var()   #feature 4                               
#   flat_image = img.flatten()                                  
#   skewness = skew(flat_image, bias = False)                  #from scipy.stats import skew, kurtosis 
#   kurt = kurtosis(flat_image, bias = False)

#   glcm1 = greycomatrix(img, distances=[1], angles=[0], symmetric=True, normed=True)
#   contrast1 = greycoprops(glcm1, 'contrast')[0][0]         # feature 5
#   energy1 = greycoprops(glcm1,'energy')[0][0]              # feature 6
#   ASM1 = greycoprops(glcm1, 'ASM')[0][0]                   # feature 7
#   homogeneity1 = greycoprops(glcm1, 'homogeneity')[0][0]   # feature 8
#   correlation1 = greycoprops(glcm1, 'correlation')[0][0]

#   glcm2 = greycomatrix(img, distances=[1], angles=[np.pi/4] , symmetric=True, normed=True)
#   contrast2 = greycoprops(glcm2, 'contrast')[0][0]         # feature 5
#   energy2 = greycoprops(glcm2,'energy')[0][0]              # feature 6
#   ASM2 = greycoprops(glcm2, 'ASM')[0][0]                   # feature 7
#   homogeneity2 = greycoprops(glcm2, 'homogeneity')[0][0]   # feature 8
#   correlation2 = greycoprops(glcm2, 'correlation')[0][0]

#   glcm3 = greycomatrix(img, distances=[1], angles=[np.pi/2], symmetric=True, normed=True)
#   contrast3 = greycoprops(glcm3, 'contrast')[0][0]         # feature 5
#   energy3 = greycoprops(glcm3,'energy')[0][0]              # feature 6
#   ASM3 = greycoprops(glcm3, 'ASM')[0][0]                   # feature 7
#   homogeneity3 = greycoprops(glcm3, 'homogeneity')[0][0]   # feature 8
#   correlation3 = greycoprops(glcm3, 'correlation')[0][0]

#   glcm4 = greycomatrix(img, distances=[1], angles=[(3*np.pi)/4], symmetric=True, normed=True)
#   contrast4 = greycoprops(glcm4, 'contrast')[0][0]         # feature 5
#   energy4 = greycoprops(glcm4,'energy')[0][0]              # feature 6
#   ASM4 = greycoprops(glcm4, 'ASM')[0][0]                   # feature 7
#   homogeneity4 = greycoprops(glcm4, 'homogeneity')[0][0]   # feature 8
#   correlation4 = greycoprops(glcm4, 'correlation')[0][0]

#   mean_contrast = (contrast1 + contrast2 + contrast3 + contrast4)/4
#   mean_energy = (energy1+ energy2+ energy3+ energy4)/4
#   mean_ASM = (ASM1+ ASM2+ ASM3+ ASM4)/4
#   mean_homogeneity = (homogeneity1+ homogeneity2+ homogeneity3+ homogeneity4)/4
#   mean_correlation = (correlation1+ correlation2+ correlation3+ correlation4)/4

#   range_contrast = max(contrast1, contrast2, contrast3, contrast4) -min(contrast1, contrast2, contrast3, contrast4)
#   range_energy = max(energy1, energy2, energy3, energy4) - min(energy1, energy2, energy3, energy4)
#   range_ASM = max(ASM1, ASM2, ASM3, ASM4) - min(ASM1, ASM2, ASM3, ASM4)
#   range_homogeneity = max(homogeneity1, homogeneity2, homogeneity3, homogeneity4) - min(homogeneity1, homogeneity2, homogeneity3, homogeneity4)
#   range_correlation = max(correlation1, correlation2, correlation3, correlation4) - min(correlation1, correlation2, correlation3, correlation4)
 
#   features = features.append({
#       'Mean': mean,
#       'variance':var, 
#       'skewness': skewness, 
#       'kurtosis': kurt,
#       'Contrast': mean_contrast,
#       'Energy': mean_energy,
#       'ASM': mean_ASM,
#       'Homogeneity': mean_homogeneity,
#       'Correlation': mean_correlation,
#       'Range_Contrast': range_contrast,
#       'Range_Energy': range_energy,
#       'Range_ASM': range_ASM,
#       'Range_Homogeneity': range_homogeneity,
#       'Range_Correlation': range_correlation}, ignore_index=True)
  
#   labels.append(2)

# path = '/content/jpgs/meningioma/*.jpg'   
# files=glob.glob(path)

# for file in files:
#   img = plt.imread(file)
#   img = np.array(img) 
#   mean=img.mean()  #feature 3
#   var=img.var()   #feature 4
#   flat_image = img.flatten()
#   skewness = skew(flat_image, bias = False)
#   kurt = kurtosis(flat_image, bias = False)

#   glcm1 = greycomatrix(img, distances=[1], angles=[0], symmetric=True, normed=True)
#   contrast1 = greycoprops(glcm1, 'contrast')[0][0]         # feature 5
#   energy1 = greycoprops(glcm1,'energy')[0][0]              # feature 6
#   ASM1 = greycoprops(glcm1, 'ASM')[0][0]                   # feature 7
#   homogeneity1 = greycoprops(glcm1, 'homogeneity')[0][0]   # feature 8
#   correlation1 = greycoprops(glcm1, 'correlation')[0][0]

#   glcm2 = greycomatrix(img, distances=[1], angles=[np.pi/4] , symmetric=True, normed=True)
#   contrast2 = greycoprops(glcm2, 'contrast')[0][0]         # feature 5
#   energy2 = greycoprops(glcm2,'energy')[0][0]              # feature 6
#   ASM2 = greycoprops(glcm2, 'ASM')[0][0]                   # feature 7
#   homogeneity2 = greycoprops(glcm2, 'homogeneity')[0][0]   # feature 8
#   correlation2 = greycoprops(glcm2, 'correlation')[0][0]

#   glcm3 = greycomatrix(img, distances=[1], angles=[np.pi/2], symmetric=True, normed=True)
#   contrast3 = greycoprops(glcm3, 'contrast')[0][0]         # feature 5
#   energy3 = greycoprops(glcm3,'energy')[0][0]              # feature 6
#   ASM3 = greycoprops(glcm3, 'ASM')[0][0]                   # feature 7
#   homogeneity3 = greycoprops(glcm3, 'homogeneity')[0][0]   # feature 8
#   correlation3 = greycoprops(glcm3, 'correlation')[0][0]

#   glcm4 = greycomatrix(img, distances=[1], angles=[(3*np.pi)/4], symmetric=True, normed=True)
#   contrast4 = greycoprops(glcm4, 'contrast')[0][0]         # feature 5
#   energy4 = greycoprops(glcm4,'energy')[0][0]              # feature 6
#   ASM4 = greycoprops(glcm4, 'ASM')[0][0]                   # feature 7
#   homogeneity4 = greycoprops(glcm4, 'homogeneity')[0][0]   # feature 8
#   correlation4 = greycoprops(glcm4, 'correlation')[0][0]

#   mean_contrast = (contrast1 + contrast2 + contrast3 + contrast4)/4
#   mean_energy = (energy1+ energy2+ energy3+ energy4)/4
#   mean_ASM = (ASM1+ ASM2+ ASM3+ ASM4)/4
#   mean_homogeneity = (homogeneity1+ homogeneity2+ homogeneity3+ homogeneity4)/4
#   mean_correlation = (correlation1+ correlation2+ correlation3+ correlation4)/4

#   range_contrast = max(contrast1, contrast2, contrast3, contrast4) -min(contrast1, contrast2, contrast3, contrast4)
#   range_energy = max(energy1, energy2, energy3, energy4) - min(energy1, energy2, energy3, energy4)
#   range_ASM = max(ASM1, ASM2, ASM3, ASM4) - min(ASM1, ASM2, ASM3, ASM4)
#   range_homogeneity = max(homogeneity1, homogeneity2, homogeneity3, homogeneity4) - min(homogeneity1, homogeneity2, homogeneity3, homogeneity4)
#   range_correlation = max(correlation1, correlation2, correlation3, correlation4) - min(correlation1, correlation2, correlation3, correlation4)
 
#   features = features.append({
#       'Mean': mean,
#       'variance':var, 
#       'skewness': skewness, 
#       'kurtosis': kurt,
#       'Contrast': mean_contrast,
#       'Energy': mean_energy,
#       'ASM': mean_ASM,
#       'Homogeneity': mean_homogeneity,
#       'Correlation': mean_correlation,
#       'Range_Contrast': range_contrast,
#       'Range_Energy': range_energy,
#       'Range_ASM': range_ASM,
#       'Range_Homogeneity': range_homogeneity,
#       'Range_Correlation': range_correlation}, ignore_index=True)
#   labels.append(1)


In [9]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest = train_test_split(features, labels, random_state = 4, test_size = .2)

from imblearn.over_sampling import SMOTE

sm = SMOTE()
xtrain, ytrain = sm.fit_sample(xtrain, ytrain)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler = scaler.fit(xtrain)
xtrain = scaler.transform(xtrain)
xtest = scaler.transform(xtest)

In [27]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
model1 = SVC(kernel = 'linear')
model1.fit(xtrain, ytrain)
print(type(model1).__name__ ," \nTesting Score", model1.score(xtest,ytest))
y_predict = model1.predict(xtest)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(ytest, y_predict))

SVC  
Testing Score 0.765807962529274
[[119  13]
 [ 87 208]]


In [32]:
from sklearn.svm import SVC

model2 = SVC(kernel = 'rbf')
model2.fit(xtrain, ytrain)
print(type(model2).__name__ ,"\nTesting Score", model2.score(xtest,ytest))
y_predict = model2.predict(xtest)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(ytest, y_predict))

SVC 
Testing Score 0.7681498829039812
[[117  15]
 [ 84 211]]


In [33]:
from sklearn.svm import SVC

model3 = SVC(kernel = 'poly')

model3.fit(xtrain, ytrain)
print(type(model3).__name__ ,"\nTesting Score", model3.score(xtest,ytest))
y_predict = model3.predict(xtest)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(ytest, y_predict))

SVC 
Testing Score 0.8032786885245902
[[118  14]
 [ 70 225]]


In [35]:
from sklearn.svm import SVC

model4 = SVC(kernel = 'poly', degree=2)

model4.fit(xtrain, ytrain)
print(type(model4).__name__ ,"\nTesting Score", model4.score(xtest,ytest))
y_predict = model4.predict(xtest)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(ytest, y_predict))

SVC 
Testing Score 0.7868852459016393
[[117  15]
 [ 76 219]]


In [34]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
for k in range(1,10):
  if k%2 != 0:
    modelknn = KNeighborsClassifier(n_neighbors=k)
    modelknn.fit(xtrain, ytrain)
    print(type(model1).__name__ ,"\n Neighbours: ",k ,"\nTesting Score", modelknn.score(xtest,ytest))
    y_predict = model1.predict(xtest)
    print(confusion_matrix(ytest, y_predict))

SVC 
 Neighbours:  1 
Testing Score 0.8407494145199064
[[119  13]
 [ 87 208]]
SVC 
 Neighbours:  3 
Testing Score 0.810304449648712
[[119  13]
 [ 87 208]]
SVC 
 Neighbours:  5 
Testing Score 0.810304449648712
[[119  13]
 [ 87 208]]
SVC 
 Neighbours:  7 
Testing Score 0.8032786885245902
[[119  13]
 [ 87 208]]
SVC 
 Neighbours:  9 
Testing Score 0.7915690866510539
[[119  13]
 [ 87 208]]
